<a href="https://colab.research.google.com/github/RaimundoLorca/roboticafcfm2021-2/blob/main/Auxiliar_7/Auxiliar_7_Pauta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Auxiliar 7: Convolutional Neural Networks
<img src="https://raw.githubusercontent.com/cherrerab/roboticafcfm/master/auxiliar_07/feature_maps.png" width="800">

Las Redes Neuronales Convolucionales (CNN) consisten en arquitecturas muy similares a las redes `Dense` o Fully Connected vistas en la auxiliar anterior, pues estás también se constituyen de de nodos con parámetros entrenables, como son los pesos y biases encargados de ponderar la información de entrada o `input.`

No obstante, la estructura de las Redes Convolucionales está diseñada particularmente para interpretar imágenes y aprender, durante su entrenamiento, a extraer patrones y características espaciales (`features`) de estas mediante la aplicación secuencial de filtros. En este sentido, este tipo de modelos han probado desempeñarse bastante bien en tareas de visión computacional, tales como reconocimiento y clasificación de objetos en imágenes, siendo esta eficacia una de las principales razones del reconocimiento del potencial del Deep Learning en las últimas décadas.

![animation](https://miro.medium.com/max/1200/1*QPRC1lcfYxcWWPAC2hrQgg.gif "animation")

A diferencia de las capas `Dense`, las capas `Convolucionales` en realidad se componen de filtros similares a los vistos en el primer capitulo de visión computacional del curso. De este modo, los pesos de la red pasan a ser estructurados como los parámetros que definen el `kernel` del filtro. Durante el procesamiento de la imágen al interior de la red, los filtros de cada capa van generando nuevas imágenes (`feature maps`), con la finalidad de ir segmentando las `features` principales que permiten aislar la información importante de los datos.

<img src="https://raw.githubusercontent.com/cherrerab/roboticafcfm/master/auxiliar_07/filtro_diagram.png" height="200">

Con la finalidad de introducir la implementación de este tipo de arquitecturas en `Tensorflow` estudiaremos un caso de regresión simple, en donde entrenaremos un modelo CNN para detectar la posición $(x, z)$ de una esfera en un espacio tridimensional.

<img src="https://raw.githubusercontent.com/cherrerab/deeplearningfallas/master/workshop_02/bin/tensorflow.png" width="400">

`TensorFlow`, en términos generales, consiste en un framework diseñado para desarrollar e implementar algoritmos de Machine Learning, y por supuesto, entre ellos, modelos de Deep Learning. Una de las particulares de este framework es que ofrece toda una gama de niveles de abstracción, desde el desarrollo de modelos de mayor complejidad mediante herramientas `low-level`, hasta la compilación y el entrenamiento de arquitecturas mediante estructuras `high-level`, como la API Keras.

Puede encontrar la documentación de estas librerías en los siguientes links:
- https://www.tensorflow.org/api_docs/python/
- https://keras.io/api/





In [ ]:
!git clone https://github.com/RaimundoLorca/roboticafcfm2021-2.git
%cd /content/roboticafcfm2021-2

## Ray Tracing Dataset

El Ray Tracing consiste en un algoritmo de renderizado de imágenes que desempeña la tarea de calcular el camino de la luz como píxeles en un plano de la imagen y simular sus efectos sobre las superficies virtuales en las que incide. En este caso, trabajaremos con un dataset que contiene imágenes de esferas en un espacio 3D y se utilizara CNN para estimar la posición de la esfera en el plano **(x,z)**


<img src="https://raw.githubusercontent.com/cherrerab/roboticafcfm/master/auxiliar_07/spheres_examples.png" height="200">

De este modo, utilizaremos este programa para generar una serie de imágenes (`samples`) de `128x128px` que contengan una esfera en distintas posiciones en el espacio. Para ahorrar tiempo y concentrarnos en el desarrollo del modelo, este proceso ya ha sido realizado y el dataset `CNN_dataset_128px.npz` correspondiente ha sido cargado a un Google Drive.

In [ ]:
!pip install -U -q PyDrive

import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# inicializar GoogleDrive con credenciales de autorización
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# crear carpeta para descargar los archivos .npz
!mkdir /content/datasets

# Google Drive IDs para descargar los archivos .npz
files_id = [('CNN_dataset_128px.npz', '1nDDvUBunpRJpTzGaQgdeRVmBTbCxa3bJ')]

# comenzar descarga
print('descargando datasets: ', end='')

for filename, id in files_id:
  save_path = os.path.join('/content/datasets', filename)

  # descargar y guardar en /content/datasets
  downloaded = drive.CreateFile({'id': id}) 
  downloaded.GetContentFile(save_path)

# indicar descarga terminada
print('done')

Podemos cargar este archivo mediante `np.load()` y explorar las estructuras y datos que contiene.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# ---
# cargar archivo CNN_dataset_128px.npz
dataset = np.load('/content/datasets/CNN_dataset_128px.npz', allow_pickle=True)

# print keys del dataset
print('dataset.keys: ',  list( dataset.keys() ) )

# ---
# extraer conjuntos de imágenes y normalizar en [0., 1.]
X = dataset['X']
X = np.float32( X/255.0 )

# extraer conjunto de posiciones (x, z)
Y = dataset['Y']

# ---
# visualizar muestra del dataset
sample_idx = np.random.choice( np.arange(X.shape[0]), 5 )
img_sample = [ X[i, :, :, :].reshape( (128, 128, 3) ) for i in sample_idx ]
img_sample = np.hstack(img_sample)

plt.figure( figsize=(12, 12) )
plt.imshow(img_sample)

print('Y:\n', Y[sample_idx, :])

---
## Data Splitting

Análogamente al caso estudiado en el workshop anterior, teniendo ya el dataset para el entrenamiento de la red neuronal, se debe dividir este en dos sets: uno de entrenamiento (`training set`) y otro de testing (`testing set`). El primero es utilizado, como su nombre lo indica, en el entrenamiento de la red neuronal, mientras que el segundo es utilizado para evaluar el desempeño del modelo ya entrenado.

El `data splitting` se puede lograr con el bloque de código a continuación, mediante la función `train_test_split` de `sklearn`.
- https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

In [ ]:
# importar librerías
from sklearn.model_selection import train_test_split

# ---
# generar sets de datos de training y testing
# la varibale test_size permite controlar la proporción entre los datos de testing y training.
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2)

# adicionalmente generaremos un conjunto de validación
# este conjunto será utilizado para monitorear la generalización del modelo
# durante el entrenamiento, sin utilizar el conjunto de testing.
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size = 0.2)

# print sample distribution
print( 'train split: {:d} samples'.format(X_train.shape[0]) )
print( '\nvalidation split: {:d} samples'.format(X_val.shape[0]) )
print( '\ntesting split: {:d} samples'.format(X_test.shape[0]) )

---
# Model Building

Para configurar nuestro modelo de regresión utilizaremos nuevamente la librería `keras` o `tf.keras`. Como ya introducimos en el workshop anterior, Keras es una API de alto nivel para la creación y el entrenamiento de modelos de deep learning. Está orientada y diseñada para la construcción de modelos de forma modular o en bloques. De este modo, ofrece un framework mucho más amigable e intuitivo para principiantes, a la vez que mantiene un estructura personalizable y versátil que permite a usuarios más avanzados incorporar nuevas ideas.

<img src="https://raw.githubusercontent.com/cherrerab/deeplearningfallas/master/workshop_02/bin/keras_logo.png" width="400">


## Model Setup

Dado que en este caso nuestros datos o `samples` consisten en imágenes a color en RGB, implementaremos una red neuronal convolucional de regresión (CNN) para procesar las imágenes sin perder la bidimensionalidad de su información.

Por lo general, los modelos CNN se componen de series de capas `keras.layers.Conv2D` junto con algún tipo de Pooling Layer, como las `keras.layers.MaxPool2D` o las `keras.layers.AveragePooling2D`. 

- https://keras.io/api/layers/convolution_layers/convolution2d/
- https://keras.io/api/layers/pooling_layers/

El propósito de las Pooling Layers es realizar un `down-sampling` de los `feature maps` generados por las capas `Conv2D` y de este modo, reducir significativamente la dimensionalidad de la información a medida que esta avanza en el modelo y alcanza mayores grados de abstracción.

<img src="https://raw.githubusercontent.com/cherrerab/roboticafcfm/master/auxiliar_07/CNN_diagram.png" height="200">

Finalmente, para completar el modelo es necesario unir la información de los `feature maps` con la capa de salida `linear` que entregará las predicciones de la posición de las esferas. Para llevar esto a cabo, el tensor de los `feature maps` es vectorizado o aplanado en una sola dimensión mediante un `keras.layers.Flatten` para luego continuar con una serie de capas `keras.layers.Dense` que se encargan de terminar el procesamiento de la información.

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Conv2D
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers import MaxPooling2D
from keras.layers import AveragePooling2D

# inicializar modelo keras.Sequential
model = Sequential()

# ---
# primero debemos agregar nuestra capa Input donde debemos especificar
# las dimensiones de los datos que se ingresarán al modelo
# las capas Conv2D reciben tensores de la forma (height, width, channels)
input_dim = ( 128, 128, 3)
model.add( Input( shape=input_dim ) )

# ---
# ahora debemos ir agregando nuestras capas Conv2D y Pooling.

# las keras.layers.Conv2D reciben la cantidad de filtros dentro de la capa,
# el tamaño de estos filtros y la función de activación con que operarán.
# https://keras.io/api/layers/convolution_layers/convolution2d/

# las keras.layers.MaxPooling2D reciben el tamaño de la ventana sobre
# la cual llevarán a cabo el down-sampling
# https://keras.io/api/layers/pooling_layers/max_pooling2d/

model.add( Conv2D(32, (5, 5), activation='relu', padding='same' ) )
model.add( Conv2D(32, (5, 5), activation='relu', padding='same' ) )

model.add( MaxPooling2D( pool_size=(2, 2) ) )

model.add( Conv2D(64, (5, 5), activation='relu', padding='same' ) )
model.add( Conv2D(64, (5, 5), activation='relu', padding='same' ) )

model.add( MaxPooling2D( pool_size=(2, 2) ) )

model.add( Conv2D(32, (5, 5), activation='relu', padding='same' ) )
model.add( Conv2D(32, (5, 5), activation='relu', padding='same' ) )

model.add( MaxPooling2D( pool_size=(2, 2) ) )

model.add( Conv2D(16, (5, 5), activation='relu', padding='same' ) )
model.add( Conv2D(16, (5, 5), activation='relu', padding='same' ) )

model.add( MaxPooling2D( pool_size=(2, 2) ) )

# ---
# ahora debemos ir agregando nuestras capas Dense para procesar la
# información hasta la capa de salida.
# https://keras.io/api/layers/core_layers/dense/

model.add( Flatten() )

model.add( Dense(units=256, activation='relu') )
model.add( Dense(units=128, activation='relu') )
model.add( Dense(units=64, activation='relu') )
model.add( Dense(units=32, activation='relu') )

# ---
# por último debemos configurar nuestra capa de salida
# dado que el modelo consiste en uno de regresión emplearemos
# la función linear, donde cada nodo indicará la predicción de posición
# de la esfera correspondiente.
labels_num = 2
model.add( Dense(units=labels_num, activation='linear') )

# print model.summary()
model.summary()

## Compile Model

Antes de poner a entrenar al modelo, es necesario realizar unas configuraciones adicionales. En particular, debemos especificar la función de pérdida o `loss function` que se optimizará durante el entrenamiento y el método de optimización como SGD o Adam.
- https://keras.io/api/models/model_training_apis/
- https://keras.io/api/optimizers/

In [ ]:
from tensorflow.keras.optimizers import Adam

# configurar optimizador Adam
# https://keras.io/api/optimizers/adam/
opt = Adam( learning_rate=1e-3 )

# ---
# compilar modelo siguiendo como función de pérdida
# el error cuadrado medio
model.compile(loss='mean_squared_error' , optimizer=opt, metrics=['mae'])

## Model Training
Hemos llegado a la parte final del proceso, para entrenar nuestro modelo debemos especificar los sets que utilizaremos para el proceso `(X_train, Y_train)`, la cantidad de `epochs` que durará el entrenamiento, y el `batch size` de muestras que se irán entregando al modelo a medida que este va iterativamente ajustando sus parámetros.

Para entrenar `keras.Models` se utiliza el método `keras.Model.fit`, el cual aparte de iniciar y realizar la rutina de entrenamiento, retorna un registro `History`. Mediante `History.history` es posible acceder a la evolución de la función de pérdida durante el entrenamiento tanto sobre los datos de `train` como sobre los de `validation`.

In [ ]:
from keras.callbacks import ModelCheckpoint
from utils import plot_loss_function

# realizar rutina de entrenamiento
model_history = model.fit(X_train, Y_train,
                          batch_size=32 , epochs=25 ,
                          validation_data=(X_val, Y_val))

# plot gráfico de función de pérdida
plot_loss_function(model_history, figsize=(10,4))

## Model Evaluation
Finalmente, una vez entrenado nuestro modelo debemos evaluar su desempeño. En este caso particular, debemos usar los datos que aislamos para `testing` `(X_test, Y_test)`. Para utilizar el `keras.Model` sobre nuevos datos de clasificación, conviene utilizar el método `keras.Sequential.predict`.

In [ ]:
from sklearn.metrics import mean_squared_error

from utils import plot_classification_map
from utils import plot_confusion_matrix

# obtener predicciones de X_test con model.predict
Y_pred = model.predict(X_test)
Y_true = Y_test

# calcular error mse de predicción
mse = mean_squared_error(Y_true, Y_pred)
print('testing mse: {:2.3f}'.format(mse))


Una forma de visualizar el proceso flujo de información a lo largo del modelo, es extraer los feature maps de cada una de las capas convolucionales. Con estos, se puede ver que tipo de features extrae cada capa y como varía a lo largo que se avanza en el modelo.

In [ ]:
from keras.models import Model
from utils import plot_img_samples

for j in range(11):
  # ---
  # compilar submodelo de la CNN
  input = model.input
  CNN_output = model.layers[j].output
  fmap_model = Model(input, CNN_output)

  # obtener dimensiones de los feature maps
  n_filters = CNN_output.shape[3]
  height, width = CNN_output.shape[1:3]

  # ---
  # extraer CNN feature maps
  img = X_train[50, :, :, :]
  x = np.reshape(img, (1, 128, 128, 3))
  CNN_fmap = fmap_model.predict(x)

  # reordenar feature maps a (feature_maps, height, width)
  fmaps = np.zeros( (n_filters, height, width) )
  for i in range( n_filters ):
    fmap = CNN_fmap[:, :, :, i]
    fmaps[i, :, :] = np.reshape( fmap, (1, height, width) )

  # visulizar fmaps mediante plot_img_samples
  plot_img_samples(fmaps, range(30), grid=(3, 10),
                  figsize=(15,15),title='CNN feature maps: ' + CNN_output.name)
